In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from pandas import Series, DataFrame

from geoband.API import *
import folium
import json
import geopandas as gpd

### 5. 오산시_연령별_거주인구격자(유소년)

In [12]:
data5 = gpd.read_file('5.오산시_연령별_거주인구격자(유소년).geojson')
data5.head()

,gid,val,geometry
0,다사551085,NaN,(POLYGON ((126.9942156468143 37.17418235770403...
1,다사551086,NaN,(POLYGON ((126.9942096381632 37.17508373885349...
2,다사551087,NaN,(POLYGON ((126.9942036292448 37.17598511986466...
3,다사551088,NaN,(POLYGON ((126.9941976200589 37.17688650073755...
4,다사552085,NaN,(POLYGON ((126.9953420754172 37.17418716043615...


In [13]:
data5.shape

(4510, 3)

In [14]:
data5.describe()

,val
count,967.000000
mean,33.707342
std,45.901711
min,0.000000
25%,0.000000
50%,13.000000
75%,54.000000
max,422.000000


In [15]:
data5.isnull().sum()

gid            0
val         3543
geometry       0
dtype: int64

In [16]:
data5 = data5.dropna(axis=0)
data5.shape

(967, 3)

In [17]:
data5.head()

,gid,val,geometry
45,다사557084,0.0,(POLYGON ((127.0009801505264 37.17330963126954...
59,다사558088,0.0,(POLYGON ((127.0020829068639 37.17691989826309...
107,다사561086,0.0,(POLYGON ((127.0054740699322 37.17513128589124...
127,다사562085,0.0,(POLYGON ((127.0066063760226 37.17423459885707...
148,다사563085,0.0,(POLYGON ((127.0077328075276 37.17423928380897...


In [18]:
index5 = data5[data5['val']==0].index
data5 = data5.drop(index5)
data5.shape

(621, 3)

In [19]:
data5.head()

,gid,val,geometry
549,다사575108,6.0,(POLYGON ((127.0211191287615 37.19502648340971...
629,다사578076,88.0,(POLYGON ((127.0246801260675 37.16619579983916...
630,다사578077,96.0,(POLYGON ((127.0246744809541 37.16709718632111...
631,다사578078,23.0,(POLYGON ((127.0246688355894 37.16799857266487...
662,다사578109,6.0,(POLYGON ((127.0244937046088 37.19594148077861...


In [20]:
from shapely.geometry import Point, shape, mapping
from shapely.ops import unary_union

max_x = []
max_y = []
min_x = []
min_y = []

for i in range(len(data5)):
    coord_x = []
    coord_y = []
    coord = mapping(data5.geometry[i])['coordinates']
    for j in range(len(coord[0][0])):
        coord_x.append(coord[0][0][j][0])
        coord_y.append(coord[0][0][j][1])
    max_x.append(max(coord_x))
    max_y.append(max(coord_y))
    min_x.append(min(coord_x))
    min_y.append(min(coord_y))

In [21]:
data5 = pd.DataFrame({'val':data5.val,
                       'max_lon':max_x,
                       'max_lat':max_y,
                       'min_lon':min_x,
                       'min_lat':min_y})
data5.head()

,val,max_lon,max_lat,min_lon,min_lat
549,6.0,127.022246,37.195932,127.021113,37.195026
629,88.0,127.025806,37.167102,127.024674,37.166196
630,96.0,127.025801,37.168003,127.024669,37.167097
631,23.0,127.025795,37.168904,127.024663,37.167999
662,6.0,127.025620,37.196847,127.024488,37.195941


### 10. 오산시_학교위치정보

In [22]:
data10 = gpd.read_file('10.오산시_학교위치정보.csv')
data10.head()

,학교구분,학교명,학교위치_경도,학교위치_위도,geometry
0,초등학교,원동초등학교,127.06969860000001,37.1351683,None
1,초등학교,운산초등학교,127.0774136,37.15249986,None
2,초등학교,화성초등학교,127.06885759999999,37.16461498,None
3,중학교,대호중학교,127.0569778,37.1618638,None
4,중학교,매홀중학교,127.0685001,37.16777784,None


In [23]:
data10.shape

(41, 5)

In [24]:
data10.describe()

,학교구분,학교명,학교위치_경도,학교위치_위도,geometry
count,41,41,41,41,0.0
unique,3,41,40,40,0.0
top,초등학교,성호고등학교,127.0613054,37.15300447,NaN
freq,24,1,2,2,NaN


In [25]:
data10.iloc[:,0].unique()

array(['초등학교', '중학교', '고등학교'], dtype=object)

In [26]:
data10.isnull().sum()

학교구분         0
학교명          0
학교위치_경도      0
학교위치_위도      0
geometry    41
dtype: int64

In [27]:
data10 = data10[(data10['학교구분']=='초등학교')]
data10.shape

(24, 5)

In [28]:
data10 = data10[['학교위치_경도','학교위치_위도']]
data10.head()

,학교위치_경도,학교위치_위도
0,127.06969860000001,37.1351683
1,127.0774136,37.15249986
2,127.06885759999999,37.16461498
12,127.04774909999999,37.14791946
13,127.090752,37.151697799999994


In [29]:
data10 = data10.drop_duplicates(['학교위치_경도','학교위치_위도'],keep='first')
data10.head()

,학교위치_경도,학교위치_위도
0,127.06969860000001,37.1351683
1,127.0774136,37.15249986
2,127.06885759999999,37.16461498
12,127.04774909999999,37.14791946
13,127.090752,37.151697799999994


In [30]:
data10.shape

(24, 2)

### 13. 오산시_어린이집_유치원현황

In [113]:
data13 = gpd.read_file('13.오산시_어린이집_유치원현황.csv')
data13.head()

,시설구분,시설타입,시설명,시설위치_경도,시설위치_위도,geometry
0,어린이집,민간,해오름어린이집,127.08433629999999,37.13238264,None
1,어린이집,가정,햇살가득어린이집,127.0418168,37.1821641,None
2,어린이집,가정,행복한아이들어린이집,127.0794942,37.15289078,None
3,어린이집,가정,휴튼사과나무어린이집,127.0496376,37.1779755,None
4,어린이집,직장,(주)아모레퍼시픽 오산어린이집,127.04025159999998,37.16531071,None


In [114]:
data13.shape

(307, 6)

In [115]:
data13.describe()

,시설구분,시설타입,시설명,시설위치_경도,시설위치_위도,geometry
count,307,307,307,307,307,0.0
unique,2,8,307,296,296,0.0
top,어린이집,가정,노블킨더유치원,127.0543087,37.1633092,NaN
freq,257,150,1,3,3,NaN


In [116]:
data13.iloc[:,0].unique()

array(['어린이집', '유치원'], dtype=object)

In [106]:
data13.isnull().sum()

시설구분          0
시설타입          0
시설명           0
시설위치_경도       0
시설위치_위도       0
geometry    307
dtype: int64

In [107]:
data13 = data13[['시설위치_경도','시설위치_위도']]
data13.head()

,시설위치_경도,시설위치_위도
0,127.08433629999999,37.13238264
1,127.0418168,37.1821641
2,127.0794942,37.15289078
3,127.0496376,37.1779755
4,127.04025159999998,37.16531071


In [109]:
data13 = data13.drop_duplicates(['시설위치_경도','시설위치_위도'],keep='first')
data13.head()

,시설위치_경도,시설위치_위도
0,127.08433629999999,37.13238264
1,127.0418168,37.1821641
2,127.0794942,37.15289078
3,127.0496376,37.1779755
4,127.04025159999998,37.16531071


In [110]:
data13.shape

(296, 2)

### 27. 오산시_도로명주소_건물

In [94]:
data27 = gpd.read_file('27.오산시_도로명주소_건물.geojson')
data27.head()

,BDTYP_CD,BULD_NM,BULD_NM_DC,BULD_SE_CD,BUL_MAN_NO,EMD_CD,GRO_FLO_CO,LNBR_MNNM,LNBR_SLNO,UND_FLO_CO,geometry
0,04001,None,None,0,23692,102,1,454,5,0,(POLYGON ((127.0871424713726 37.15535053684245...
1,21005,None,None,0,18171,112,1,519,2,0,"(POLYGON ((127.0108543392694 37.1889515432221,..."
2,21005,None,None,0,18154,114,1,58,1,0,(POLYGON ((127.0358831962406 37.17901795122366...
3,15006,None,None,0,18491,114,1,58,1,0,"(POLYGON ((127.035477945367 37.17859621399108,..."
4,21005,None,None,0,18170,107,1,42,1,0,(POLYGON ((127.0386711133019 37.17609709938915...


In [95]:
data27.shape

(16449, 11)

In [96]:
data27['BDTYP_CD'].describe()

count     16449
unique      150
top       01001
freq       6399
Name: BDTYP_CD, dtype: object

In [97]:
data27.iloc[:,0].unique()

array(['04001', '21005', '15006', '15002', '13000', '04000', '03999',
       '03000', '16005', '04999', '15001', '03001', '14000', '01001',
       '17000', '16006', '04006', '16002', '14999', '02004', '02001',
       '10101', '06301', '04005', '13100', '15000', '14001', '16000',
       '08999', '04402', '02003', '02002', '01003', '08201', '04010',
       '06205', '04499', '08000', '08101', '08300', '02007', '02005',
       '02006', '16999', '08400', '17999', '18004', '05301', '10199',
       '17203', '05000', '04201', '08203', '08005', '10000', '06203',
       '11000', '07000', '06299', '07101', '03101', '10102', '19000',
       '06999', '07999', '04002', '15009', '05501', '03002', '08001',
       '08299', '10299', '10201', '10202', '16009', '09000', '04199',
       '05999', '07201', '05103', '18000', '14002', '05106', '05102',
       '16001', '09999', '20002', '04403', '08199', '03005', '08102',
       '04107', '21999', '08004', '16007', '01002', '03103', '06305',
       '11199', '081

08601(청소년수련원)
08602(유스호스텔)
08603(청소년문화의집)
08701(청소년수련원)
08702(청소년야영장)
21003(어린이회관)

In [98]:
data27 = data27[(data27['BDTYP_CD']=='08601')|(data27['BDTYP_CD']=='08602')|(data27['BDTYP_CD']=='08603')
                |(data27['BDTYP_CD']=='08701')|(data27['BDTYP_CD']=='08702')|(data27['BDTYP_CD']=='21003')]
data27.shape

(0, 11)

### 29. 오산시_체육시설현황

In [68]:
data29 = gpd.read_file('29.오산시_체육시설현황.csv')
data29.head()

,시설구분명,설치위치_경도,설치위치_위도,geometry
0,육상시설,127.0738045,37.157196500000005,None
1,기타시설,127.04192330000001,37.17944,None
2,기타시설,127.0757197,37.1587541,None
3,테니스,127.07744459999999,37.1498057,None
4,테니스,127.0702232,37.155639799999996,None


In [69]:
data29.shape

(145, 4)

In [70]:
data29.describe()

,시설구분명,설치위치_경도,설치위치_위도,geometry
count,145,145,145,0.0
unique,6,123,125,0.0
top,체육도장업,127.060368,37.161143200000005,NaN
freq,131,3,3,NaN


In [71]:
data29.iloc[:,0].unique()

array(['육상시설', '기타시설', '테니스', '수영장', '구기체육관', '체육도장업'], dtype=object)

In [72]:
index29 = data29[data29['시설구분명']=='테니스'].index
data29 = data29.drop(index29)
data29.shape

(139, 4)

In [73]:
data29.isnull().sum()

시설구분명         0
설치위치_경도       0
설치위치_위도       0
geometry    139
dtype: int64

In [74]:
data29 = data29[['설치위치_경도','설치위치_위도']]
data29.head()

,설치위치_경도,설치위치_위도
0,127.0738045,37.157196500000005
1,127.04192330000001,37.17944
2,127.0757197,37.1587541
9,127.0757197,37.1587541
10,127.0757197,37.1587541


In [75]:
data29 = data29.drop_duplicates(['설치위치_경도','설치위치_위도'],keep='first')
data29.head()

,설치위치_경도,설치위치_위도
0,127.0738045,37.157196500000005
1,127.04192330000001,37.17944
2,127.0757197,37.1587541
11,127.06969860000001,37.1351683
12,127.0422832,37.178638


In [76]:
data29.shape

(124, 2)

### 30. 오산시_학원_및 _교습소_현황.csv

In [52]:
data30 = gpd.read_file('30.오산시_학원_및_교습소_현황.csv')
data30.head()

,업종구분명,교습과정명,시설위치_경도,시설위치_위도,geometry
0,교습소,음악,127.0489187,37.1462791,None
1,교습소,미술,127.0489187,37.1462791,None
2,교습소,보습,127.052419,37.14580915,None
3,학교교과교습학원,"종합(국,수,사,과,논술)",127.0489187,37.1462791,None
4,학교교과교습학원,피아노(체르니40이상),127.0489187,37.1462791,None


In [53]:
data30.shape

(1786, 5)

In [54]:
data30.describe()

,업종구분명,교습과정명,시설위치_경도,시설위치_위도,geometry
count,1786,1786,1786,1786,0.0
unique,3,1121,197,198,0.0
top,학교교과교습학원,보습,127.0737774,37.14020279,NaN
freq,1590,34,63,63,NaN


In [55]:
data30.iloc[:,0].unique()

array(['교습소', '학교교과교습학원', '평생직업교육학원'], dtype=object)

In [56]:
index30 = data30[data30['업종구분명']=='평생직업교육학원'].index
data30 = data30.drop(index30)
data30.shape

(1680, 5)

In [57]:
data30.isnull().sum()

업종구분명          0
교습과정명          0
시설위치_경도        0
시설위치_위도        0
geometry    1680
dtype: int64

In [59]:
data30 = data30[['시설위치_경도','시설위치_위도']]
data30.head()

,시설위치_경도,시설위치_위도
0,127.0489187,37.1462791
1,127.0489187,37.1462791
2,127.052419,37.14580915
3,127.0489187,37.1462791
4,127.0489187,37.1462791


In [66]:
data30 = data30.drop_duplicates(['시설위치_경도','시설위치_위도'],keep='first')
data30.head()

,시설위치_경도,시설위치_위도
0,127.0489187,37.1462791
2,127.052419,37.14580915
7,127.0460267,37.14755968
14,127.0685281,37.13511489
15,127.0687522,37.13392538


In [67]:
data30.shape

(192, 2)